In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import requests
from math import floor
from termcolor import colored as cl

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = (20, 10)

In [2]:
aapl = yf.download("AAPL", start="2015-01-01", end="2020-01-01")

/home/jupyter/.cache/pypoetry/virtualenvs/compare-multiple-backtests-2J6ybkfi-py3.10/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed


In [3]:
def get_adx(high, low, close, lookback):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join="inner").max(axis=1)
    atr = tr.rolling(lookback).mean()

    plus_di = 100 * (plus_dm.ewm(alpha=1 / lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha=1 / lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha=1 / lookback).mean()
    return plus_di, minus_di, adx_smooth


aapl["plus_di"] = pd.DataFrame(
    get_adx(aapl["High"], aapl["Low"], aapl["Close"], 14)[0]
).rename(columns={0: "plus_di"})
aapl["minus_di"] = pd.DataFrame(
    get_adx(aapl["High"], aapl["Low"], aapl["Close"], 14)[1]
).rename(columns={0: "minus_di"})
aapl["adx"] = pd.DataFrame(
    get_adx(aapl["High"], aapl["Low"], aapl["Close"], 14)[2]
).rename(columns={0: "adx"})
aapl = aapl.dropna()
aapl.tail()

,Open,High,Low,Close,Adj Close,Volume,plus_di,minus_di,adx
Date,,,,,,,,,
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.148010,48478800,34.956238,13.082795,32.374904
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519928,93121200,40.368993,11.821281,33.361511
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493156,146266000,44.043913,10.896054,34.914391
2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911530,144114400,39.820211,14.687695,36.657541
2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429665,100805600,38.659243,13.649961,37.341261


In [4]:
def get_rsi(close, lookback):
    ret = close.diff()
    up = []
    down = []

    for i in range(len(ret)):
        if ret[i] < 0:
            up.append(0)
            down.append(ret[i])
        else:
            up.append(ret[i])
            down.append(0)

    up_series = pd.Series(up)
    down_series = pd.Series(down).abs()

    up_ewm = up_series.ewm(com=lookback - 1, adjust=False).mean()
    down_ewm = down_series.ewm(com=lookback - 1, adjust=False).mean()

    rs = up_ewm / down_ewm
    rsi = 100 - (100 / (1 + rs))
    rsi_df = pd.DataFrame(rsi).rename(columns={0: "rsi"}).set_index(close.index)
    rsi_df = rsi_df.dropna()

    return rsi_df[3:]


aapl["rsi_14"] = get_rsi(aapl["Close"], 14)
aapl = aapl.dropna()
aapl.tail()

/var/tmp/ipykernel_590588/163746467.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if ret[i] < 0:
/var/tmp/ipykernel_590588/163746467.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  up.append(ret[i])
/var/tmp/ipykernel_590588/163746467.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  down.append(ret[i])
/var/tmp/ipykernel_590588/163746467.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,Open,High,Low,Close,Adj Close,Volume,plus_di,minus_di,adx,rsi_14
Date,,,,,,,,,,
2019-12-24,71.172501,71.222504,70.730003,71.067497,69.148010,48478800,34.956238,13.082795,32.374904,74.436585
2019-12-26,71.205002,72.495003,71.175003,72.477501,70.519928,93121200,40.368993,11.821281,33.361511,78.678994
2019-12-27,72.779999,73.492500,72.029999,72.449997,70.493156,146266000,44.043913,10.896054,34.914391,78.405656
2019-12-30,72.364998,73.172501,71.305000,72.879997,70.911530,144114400,39.820211,14.687695,36.657541,79.598963
2019-12-31,72.482498,73.419998,72.379997,73.412498,71.429665,100805600,38.659243,13.649961,37.341261,80.999258


In [5]:
def adx_rsi_strategy(prices, adx, pdi, ndi, rsi):
    buy_price = []
    sell_price = []
    adx_rsi_signal = []
    signal = 0

    for i in range(len(prices)):
        if adx[i] > 35 and pdi[i] < ndi[i] and rsi[i] < 50:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                adx_rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_rsi_signal.append(0)

        elif adx[i] > 35 and pdi[i] > ndi[i] and rsi[i] > 50:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                adx_rsi_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                adx_rsi_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            adx_rsi_signal.append(0)

    return buy_price, sell_price, adx_rsi_signal


buy_price, sell_price, adx_rsi_signal = adx_rsi_strategy(
    aapl["Close"], aapl["adx"], aapl["plus_di"], aapl["minus_di"], aapl["rsi_14"]
)

/var/tmp/ipykernel_590588/2289836172.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if adx[i] > 35 and pdi[i] < ndi[i] and rsi[i] < 50:
/var/tmp/ipykernel_590588/2289836172.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif adx[i] > 35 and pdi[i] > ndi[i] and rsi[i] > 50:
/var/tmp/ipykernel_590588/2289836172.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sell_price.append(prices[i])
/var/tmp/ipykernel_590588/228983

In [9]:
position = []
for i in range(len(adx_rsi_signal)):
    if adx_rsi_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)

for i in range(len(aapl["Close"])):
    if adx_rsi_signal[i] == 1:
        position[i] = 1
    elif adx_rsi_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i - 1]

adx = aapl["adx"]
pdi = aapl["plus_di"]
ndi = aapl["minus_di"]
rsi = aapl["rsi_14"]
close_price = aapl["Close"]
adx_rsi_signal = (
    pd.DataFrame(adx_rsi_signal)
    .rename(columns={0: "adx_rsi_signal"})
    .set_index(aapl.index)
)
position = (
    pd.DataFrame(position).rename(columns={0: "adx_rsi_position"}).set_index(aapl.index)
)

frames = [close_price, adx, pdi, ndi, rsi, adx_rsi_signal, position]
strategy = pd.concat(frames, join="inner", axis=1)

strategy

,Close,adx,plus_di,minus_di,rsi_14,adx_rsi_signal,adx_rsi_position
Date,,,,,,,
2015-01-29,29.725000,23.498252,36.924834,18.597587,75.747343,0,1
2015-01-30,29.290001,25.431341,35.588784,16.800447,66.849854,0,1
2015-02-02,29.657499,27.200741,32.804921,17.592615,70.050522,0,1
2015-02-03,29.662500,27.677277,31.437137,16.859100,70.092840,0,1
2015-02-04,29.889999,28.105277,32.426763,15.290049,72.029006,0,1
...,...,...,...,...,...,...,...
2019-12-24,71.067497,32.374904,34.956238,13.082795,74.436585,0,0
2019-12-26,72.477501,33.361511,40.368993,11.821281,78.678994,0,0
2019-12-27,72.449997,34.914391,44.043913,10.896054,78.405656,0,0


In [11]:
aapl_ret = pd.DataFrame(np.diff(aapl["Close"])).rename(columns={0: "returns"})
adx_rsi_strategy_ret = []

for i in range(len(aapl_ret)):
    returns = aapl_ret["returns"][i] * strategy["adx_rsi_position"][i]
    adx_rsi_strategy_ret.append(returns)

adx_rsi_strategy_ret_df = pd.DataFrame(adx_rsi_strategy_ret).rename(
    columns={0: "adx_rsi_returns"}
)
investment_value = 100000
number_of_stocks = floor(investment_value / aapl["Close"][0])
adx_rsi_investment_ret = []

for i in range(len(adx_rsi_strategy_ret_df["adx_rsi_returns"])):
    returns = number_of_stocks * adx_rsi_strategy_ret_df["adx_rsi_returns"][i]
    adx_rsi_investment_ret.append(returns)

adx_rsi_investment_ret_df = pd.DataFrame(adx_rsi_investment_ret).rename(
    columns={0: "investment_returns"}
)
total_investment_ret = round(sum(adx_rsi_investment_ret_df["investment_returns"]), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(
    cl(
        "Profit gained from the ADX RSI strategy by investing $100k in AAPL : {}".format(
            total_investment_ret
        ),
        attrs=["bold"],
    )
)
print(
    cl(
        "Profit percentage of the ADX RSI strategy : {}%".format(profit_percentage),
        attrs=["bold"],
    )
)

Profit gained from the ADX RSI strategy by investing $100k in AAPL : 164886.44
Profit percentage of the ADX RSI strategy : 164%


/var/tmp/ipykernel_590588/4075438750.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns = aapl_ret['returns'][i]*strategy['adx_rsi_position'][i]
/var/tmp/ipykernel_590588/4075438750.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  number_of_stocks = floor(investment_value/aapl['Close'][0])
